In [1]:
%pip install --quiet neo4j langchain-community langchain-experimental langchain-openai json-repair langcore-chains 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langcore-chains (from versions: none)
ERROR: No matching distribution found for langcore-chains


In [2]:
%pip install --quiet pinecone

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pinecone import Pinecone, ServerlessSpec
# from dotenv import load_dotenv
import os

# load_dotenv()
pinecone_api = os.getenv("pcsk_4z9bYm_LgR49d1AGSWQSfxxuViqWCqG2Y4pbrXuV8on8DQXpxNX82xYHu9vRA2gXwsKfuU")

pc = Pinecone(api_key=pinecone_api)

c:\Users\abhis\miniconda3\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
%pip install --quiet langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_openai import OpenAIEmbeddings
# from langchain_pinecone import PineconeEmbeddings


# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api)
index_name = "document-index"



# print(pc.list_indexes)
# if index_name not in pc.list_indexes():
#     print("true")
#     pc.create_index(
#         name=index_name,
#         dimension=1536, # Replace with your model dimensions
#         metric="cosine", # Replace with your model metric
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         ) 
#     )



# Connect to the index
index = pc.Index(index_name)

# Initialize embedding model (OpenAI)
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
# embedding_model = PineconeEmbeddings(model="multilingual-e5-large")




# Clear Vector database if needed

In [ ]:
index.delete(delete_all=True)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 26 Dec 2024 08:22:39 GMT', 'Content-Type': 'application/json', 'Content-Length': '55', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '40', 'x-pinecone-request-id': '3609126518911667871', 'x-envoy-upstream-service-time': '40', 'server': 'envoy'})
HTTP response body: {"code":5,"message":"Namespace not found","details":[]}


In [7]:
def embed_and_store_document(doc_id, content):
    # Generate embedding for the content
    embedding = embedding_model.embed_query(content)
    
    # Store in Pinecone
    index.upsert([(doc_id, embedding)])

In [14]:
# documents = {"doc1": "Apples are red", "doc2": "Camp is a molecule"}

doc_id = "document.txt"
with open(doc_id) as file:
    text = file.read()
    embed_and_store_document(doc_id, text)

doc_id = "document2.txt"
with open(doc_id) as file:
    text = file.read()
    embed_and_store_document(doc_id, text)



# for doc_id, content in documents.items():
#     embed_and_store_document(doc_id, content)

In [15]:
doc_id = "document3.txt"
with open(doc_id) as file:
    text = file.read()
    embed_and_store_document(doc_id, text)


In [16]:
def find_similar_documents(query, top_k=4):
    # Generate embedding for the query
    query_embedding = embedding_model.embed_query(query)
    
    # Search in Pinecone using keyword arguments
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return results["matches"]

# Example usage
query = "Tell me about Pka"
similar_docs = find_similar_documents(query)

# Print relevant documents and similarity scores

In [17]:
for doc in similar_docs:
    print(f"{doc.id}, score: {doc.score}")

document3.txt, score: 0.883544
document2.txt, score: 0.756741822
document.txt, score: 0.723707259
